# 深層学習ノートブック-2 自動微分(Autograd)
* pytorchでは計算グラフを自動的に構築し，勾配を逆方向に計算する  
* tensor生成時にrequires_grad=Trueを設定することで自動微分を有効にする  
* 計算後に.backward()を呼ぶことで自動的に勾配が計算される  
* 勾配情報は.grad属性に累積される  →詳細は例２
* 末端ノードに対する勾配しか保存されない
    * 中間ノードに対する勾配を保存したい場合は当該tensorに対して.retain_grad()を実行する  


## 例１

In [2]:
# import torchでpytorchをimport
import torch

# torchで使う疑似乱数のseedは設定
torch.manual_seed(42)

In [3]:
# tensorを作成し、requires_grad=Trueを設定して演算を追跡
x = torch.ones(2, 2, requires_grad=True)

# tensorに対する捜査
y = x + 2
z = y * y * 3
out = z.mean()

# 勾配を計算
out.backward()

# 勾配d(out)/dxを出力
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


[自分用メモ書き]  
上記では数学で求めるような導関数の計算過程なしに微分係数が求まっているので、  
数学の感覚からはちょっとわかりづらかったが、やっていることは下記。  
数学的に計算すると確かに上記のようになる。  
（ただし、y * yの部分は行列積ではなくアダマール積（対応要素同士の積）であることに注意。）  

1. tensorのxを下記のようにおく  
    $\bm{X} = \left(
    \begin{matrix} 
    x_{1} & x_{2} \\ 
    x_{3} & x_{4} \\
    \end{matrix} 
    \right)$

2. すると、上記のy,z,outは下記のように書ける。  
    $\bm{Y} = \left(
    \begin{matrix} 
    x_{1}+2 & x_{2}+2 \\ 
    x_{3}+2 & x_{4}+2 \\
    \end{matrix} 
    \right)$  

    $\bm{Z} = \left(
    \begin{matrix} 
    3(x_{1}+2)^2 & 3(x_{2}+2)^2 \\ 
    3(x_{3}+2)^2 & 3(x_{4}+2)^2 \\
    \end{matrix} 
    \right)$

    $\bm{out} = \frac{3}{4}\sum_{i=1}^{n}(x_i + 2)^2 \quad $


3. 試しにoutを$x_1$で偏微分すると、  
    $\frac{\partial out }{\partial x_1} = \frac{3}{2}(x_1+2)$  

    この$x_1$に上記例の1を代入すると最終的な結果の4.5に一致する。  
    すなわち、x.gradの結果を数学的に見ると、  
    計算過程で導関数に相当するものを求め、そこへ入力tensor(x)を代入し、各要素に対応する偏微分係数を返している模様。  


## 例2

In [4]:
# pytorchでは基本的にfloatで数値を扱う。下手にintを使うとエラーになるので注意。
x = torch.tensor([[2.0]], requires_grad=True)
y = torch.tensor([[3.0]], requires_grad=True)

# zを計算。numpyのようにpytorchでもlog,sinはある。
z = y * torch.log(x) + torch.sin(y)

# 勾配を計算
z.backward()

# zに対するx,yの偏微分を計算
print(f'dz/dx: {x.grad}')
print(f'dz/dy: {y.grad}')

dz/dx: tensor([[1.5000]])
dz/dy: tensor([[-0.2968]])


もう一回実行すると。。。

In [5]:
z = y * torch.log(x) + torch.sin(y)

# 勾配を計算
z.backward()

# zに対するx,yの偏微分を計算
print(f'dz/dx: {x.grad}')
print(f'dz/dy: {y.grad}')

dz/dx: tensor([[3.]])
dz/dy: tensor([[-0.5937]])


複数回zの計算をすると、x.grad,y.gradが累積されていくので注意。

## 例3 中間ノードの勾配

$z=(x+y)^2$の末端ノードにおける勾配は$\partial{z}/\partial{x}$、$\partial{z}/\partial{y}$であり、  
これは.backwardを使えば自動的に求まる。  
一方、中間ノード$\partial{z}/\partial{(x+y)}$の勾配は途中で.retain_grad()を使って明示的に保持させる必要がある。

In [6]:
# 2.0は2.とも書いてOK。
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)
a = x + y

# 中間ノードの勾配も求めたい場合は最終的な計算結果（z）の前にretain_grad（）を実行する。
a.retain_grad()

# zを計算。numpyのようにpytorchでもlog,sinはある。
z = a ** 2

# 勾配を計算
z.backward()

# zに対するx,yの偏微分を計算
print(f'dz/d(x+y): {a.grad}')
print(f'dz/dx: {x.grad}')
print(f'dz/dy: {y.grad}')

dz/d(x+y): 10.0
dz/dx: 10.0
dz/dy: 10.0


## 例4 勾配を保持する必要がない場合
* torch.no_grad()
  * Autogradで勾配を保持するには計算グラフを構築するため，計算量が高くなりメモリ使用量も増える
  * 勾配を計算する必要がないケースでは，with torch.no_grad():を使って勾配を計算しないようにする
  * 計算速度が向上し，メモリ使用量が減少する
  * モデルの推論(予測)時やパラメータの更新時に使用する

In [7]:
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)

# 定義の際にrequires_gradとした変数を使った計算でいちいち勾配を計算して欲しくないときは下記のように書く
with torch.no_grad():
    z1 = y * torch.log(x) + torch.sin(y)


z1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

no_gradの中でz1を計算したため、勾配を算出しようとするとエラーになる。